In [1]:
!cat .env

STAGE=PRODUCTION


In [6]:
import json
import os
import pandas as pd
import pytz
import random
import string
import math
import datetime
import locale
import json

from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from openpyxl import load_workbook
from datetime import timedelta, datetime
from openpyxl_image_loader import SheetImageLoader
from copy import deepcopy
from json_logic import jsonLogic

from django.contrib.gis.geos import Point
from django.contrib.admin.utils import flatten

from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history

from helpers.km_converter import get_road_coordinates
from helpers.apps.inventory import get_api_name
from helpers.strings import to_snake_case
from helpers.serializers import get_obj_serialized
from helpers.strings import get_obj_from_path
from helpers.apps.reportings import form_fields_dict

In [2]:
mytz=pytz.timezone("America/Sao_Paulo")

In [3]:
def get_value_direction_lane(company, name, value):
    select_options = company.custom_options['reporting']['fields'][name]['select_options']['options']
    translation={item['value']: item['name'] for item in select_options}
    
    return translation[value]

In [3]:
filename = 'Inventario (simplificado)_TOR'

In [4]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

In [5]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

In [6]:
values[0]

{'': None,
 'Número': 'EIXO-IT-2021.00125',
 'Rodovia': 'SP-225',
 'km inicial': 166.5,
 'km final': 166.5,
 'Lote': 'Itirapina',
 'Latitude': -22.26566295677719,
 'Longitude': -48.443926761446384,
 'de Natureza': 'Inventário',
 'de Classe': 'Sinalização vertical',
 'para Natureza': 'Sinalização Vertical',
 'para Classe': 'Reparo ou substituição das demais placas de sinalização vertical (ausência, retro inadequada, avaria, depredação ou vandalismo)',
 'Comprimento': 0,
 'Largura': 0,
 'Espessura': 0,
 'Faixa': 'Acostamento',
 'Sentido': 'Leste',
 'Status': 'Executado',
 'Criado por': ' ',
 'Equipe': 'Corpo Técnico',
 'Criado em': datetime.datetime(2021, 4, 8, 13, 29, 51, 751000),
 'Atualizado em': datetime.datetime(2021, 4, 8, 13, 29, 51, 839000),
 'Executado em': ''}

In [4]:
company=Company.objects.get(name='Eixo SP')
user=User.objects.get(username='rlcs')

In [5]:
for a in tqdm(values):
    reporting=Reporting.objects.get(company=company, number=a['Número'])
    reporting.occurrence_type=OccurrenceType.objects.get(company=company, name=a['para Classe'])
    reporting._history_user=user
#     reporting.save()

NameError: name 'values' is not defined

In [ ]:
reps=[]

for a in tqdm(values):
    reporting=Reporting.objects.get(company=company, number=a['Número'])
    reporting.form_data['demand_origin'] = [str(a['para Origem'])]
#     reporting.form_data['artesp_code'] = a['para Código ARTESP']
#     reporting.form_data['received_date'] = a['para Recebido da ARTESP'].strftime('%Y-%m-%dT%H:%M:00.000000-03:00')
#     reporting.form_data['nc_artesp'] = a['para NC ARTESP']
    reporting._history_user=user
    
#     reporting.save()

In [4]:
company=Company.objects.get(name='Eixo SP')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Executado')
reportings=Reporting.objects.filter(company=company, status=status).only('number', 'occurrence_type', 'form_data', 'due_at', 'executed_at')
user=User.objects.get(username='rlcs')

reportings.count()

In [7]:
reps=[]

for a in tqdm(reportings):
    if 'demand_origin' in a.form_data:
        if '3' in a.form_data['demand_origin'] or '4' in a.form_data['demand_origin']:
            if a.due_at is not None:
                try:
                    if a.executed_at > a.due_at:
                        a.executed_at = a.due_at
#                         print(a.number)
#                         reps.append(a)
                except Exception:
                    a.executed_at = a.due_at
#                     reps.append(a)
#                     print(a.number)

  0%|          | 0/68677 [00:00<?, ?it/s]

In [ ]:
reporting_edit=bulk_update_with_history(reps, Reporting, user=user, use_django_bulk=True, batch_size=100)

In [ ]:
for a in Reporting.objects.filter(road_name='SPA-498/294'):
    ponto, rodovia = get_road_coordinates(a.road_name, a.km, a.direction, company)
    a.point=ponto
    a._history_user=user
#     a.save()

In [13]:
reportings=Reporting.history.model.objects.filter(company=company, created_at__gt='2021-05-31', created_by__username__icontains='reginaldo', history_type='-')
reportings.count()

/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.created_at received a naive datetime (2021-05-31 00:00:00) while time zone support is active.
  RuntimeWarning)


29

In [ ]:
for a in reportings:
    print(f'{a.number} \t {a.created_at.strftime("%d/%m/%Y")} \t {a.history_date.strftime("%d/%m/%Y")} \t {User.objects.get(uuid=a.history_user_id).get_full_name()} \t {a.road_name} \t {a.km} \t {OccurrenceType.objects.get(uuid=a.occurrence_type_id).name}')

In [11]:
reporting_list=['EIXO-DRE-2021.10801', 'EIXO-DRE-2021.10768', 'EIXO-DRE-2021.10764', 'EIXO-DRE-2021.10762', 'EIXO-DRE-2021.10482',
                'EIXO-DRE-2021.10481', 'EIXO-DRE-2021.09723', 'EIXO-DRE-2021.09722', 'EIXO-DRE-2021.09673', 'EIXO-DRE-2021.04194']

reporting_deleted = Reporting.history.model.objects.filter(company=company, history_type='-', number__in=reporting_list)

In [ ]:
for a in reporting_list:
    reporting=Reporting.objects.get(company=company, number=a)
#     print(a)
#     reporting.instance.save()
    for b in ReportingFile.history.model.objects.filter(reporting=reporting.uuid): 
        try:
            print(b)
#             b.instance.save()
        except:
            print('AQUI')
            pass

In [6]:
inventory=Reporting.objects.filter(company=company, occurrence_type__occurrence_kind='2')
inventory.count()

10795

In [5]:
reps=[]
status=ServiceOrderActionStatus.objects.get(companies=company, name='Executado')

for a in tqdm(inventory):
    a.status=status
    reps.append(a)

  0%|          | 0/10788 [00:00<?, ?it/s]

In [6]:
update_inventory=bulk_update_with_history(reps, Reporting, user=user, use_django_bulk=True, batch_size=1000)

In [7]:
reportings=Reporting.objects.filter(company=company, lot__isnull=True)
reportings.count()

327

In [ ]:
for a in tqdm(reportings):
    a.save()

In [23]:
reporting=Reporting.objects.get(company=company, number='EIXO-default-2021.35797')
reporting

<Reporting: [Eixo SP] EIXO-default-2021.35797 - 2021-09-08 11:59:45.419000+00:00>

In [24]:
lista=["EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.07.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.06.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.06 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.06 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.06 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.05.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.05 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.05 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.05 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.04.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.04 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.04 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.03.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.03 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.02.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.02 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.01.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.01 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.01 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.00.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.38.00 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.59.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.59 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.58.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.57.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.57 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.57 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.56.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.56 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.56 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.55.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.55 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.54.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.54 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.54 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.53.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.53 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.53 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.52.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.51.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.51 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.50.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.50 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.49.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.49 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.48.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.47.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.47 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.47 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.46.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.46 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.45.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.45 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.45 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.44.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.44 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.44 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.43.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.43 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.42.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.42 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.41.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.41 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.40.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.40 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.40 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.40 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.39.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.39 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.38.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.37.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.36.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.36 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.36 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.36 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.35.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.35 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.35 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.35 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.34.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.33.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.33 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.32.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.31.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.31 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.31 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.31 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.30.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.30 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.30 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.29.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.29 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.28.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.28 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.28 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.27.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.25.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.25 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.25 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.24.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.24 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.24 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.23.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.22.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.22 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.21.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.21 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.20.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.20 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.20 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.19.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.19 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.19 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.19 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.18.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.18 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.17.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.17 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.17 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.15.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.14.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.13.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.13 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.13 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.13 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.12.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.11.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.11 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.10.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.10 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.09.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.09 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.08.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.07.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.07 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.07 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.06.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.06 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.06 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.05.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.05 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.04.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.04 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.03.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.02.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.02 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.02 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.01.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.01 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.00.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.37.00 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.59.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.58.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.58 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.58 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.58 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.57.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.57 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.57 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.56.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.55.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.55 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.54.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.54 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.54 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.54 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.53.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.52.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.51.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.51 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.50.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.49.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.49 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.49 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.48.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.48 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.47.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.47 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.46.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.46 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.45.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.45 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.45 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.44.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.42.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.42 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.42 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.42 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.41.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.40.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.40 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.40 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.39.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.39 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.38.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.37.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.37 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.37 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.36.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.36 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.35.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.35 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.35 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.34.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.33.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.33 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.33 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.32.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.32 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.32 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.31.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.31 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.31 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.30.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.30 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.30 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.30 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.28.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.27.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.27 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.26.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.25.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.25 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.25 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.24.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.24 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.23.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.22.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.22 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.22 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.21.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.20.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.20 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.20 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.19.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.19 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.19 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.18.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.18 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.17.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.17 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.17 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.17 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.16.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.16 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.15.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.15 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.14.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.14 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.14 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.13.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.13 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.13 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.12.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.11.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.11 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.11 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.11 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.10.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.10 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.09.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.09 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.08.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.07.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.07 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.06.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.06 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.05.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.05 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.05 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.04.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.04 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.04 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.03.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.02.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.02 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.01.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.01 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.01 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.00.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.36.00 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.59.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.59 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.59 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.58.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.57.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.56.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.56 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.56 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.56 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.55.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.55 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.54.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.54 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.53.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.53 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.52.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.51.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.51 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.50.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.49.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.49 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.49 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.48.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.48 (3).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.48 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.48 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.47.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.46.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.45.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.44.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.44 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.43.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.43 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.42.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.42 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.41.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.41 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.40.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.40 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.39.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.39 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.39 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.38.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.38 (2).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.38 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.37.jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.37 (1).jpeg",
"EIXO-default-2021.35797_WhatsApp Image 2021-09-22 at 13.35.34.jpeg"]

In [25]:
images_bulk=[]

for a in lista:
    images_bulk.append(ReportingFile(
            created_by=user,
            reporting=reporting,
            description="",
            upload=a,
            datetime=datetime.datetime(2021, 9, 8).replace(tzinfo=pytz.timezone('America/Sao_Paulo'))
        ))

In [ ]:
images_bulk

In [26]:
images_reporting=bulk_create_with_history(images_bulk, ReportingFile, batch_size=100)

In [4]:
kinds=[2, 5]
types=['Acesso', 'Principal', 'Interligação']

In [14]:
reps=[]

for reporting in tqdm(reportings):
    
#     if reporting.form_metadata == {}:
    
    fields=[a for a, b in reporting.form_metadata.items() if b["manually_specified"] == True]
    autofill_fields = [get_api_name(a) for a in reporting.occurrence_type.form_fields['fields'] if 'autofill' in a]
    reporting.form_metadata = {
            to_snake_case(field_name): { "manually_specified": True if to_snake_case(field_name) in fields else False} for field_name in autofill_fields
    }
    
    
    
    obj_serialized = get_obj_serialized(reporting, is_reporting=True)
    form_fields = form_fields_dict(reporting.occurrence_type)
    
    for key, value in reporting.form_metadata.items():
        manually_specified = get_obj_from_path(value, "manually_specified")
        
        if (isinstance(manually_specified, bool) and not manually_specified):
            form_field = get_obj_from_path(form_fields, key)
            
            if isinstance(form_field, dict):
                autofill = form_field.get("autofill", {})
                try:
                    reporting.form_data[key] = jsonLogic(autofill, obj_serialized)
                except Exception as e:
                    print(reporting.number, e, autofill)
                    pass
    
    
    reps.append(reporting)

  0%|          | 0/19 [00:00<?, ?it/s]

In [15]:
reporting_edit=bulk_update_with_history(reps, Reporting, user=user, use_django_bulk=True, batch_size=1000)

In [9]:
reportings=Reporting.history.model.objects.filter(road_name='SP-294', km__gte=423, km__lte=424, history_type='-')
reportings.count()

49

In [6]:
for a in reportings:
    print(f'Apontamento número {a.number}, Classe {a.occurrence_type.name}, no Km {a.km}')

Apontamento número EIXO-default-2021.13426, Classe Reparo ou substituição das demais placas de sinalização vertical (ausência, retro inadequada, avaria, depredação ou vandalismo), no Km 423.3
Apontamento número EIXO-CON-2020.01755, Classe Limpeza, lavagem ou pintura de barreira de concreto, no Km 424.0
Apontamento número EIXO-default-2020.03432, Classe Reparo ou substituição de sinalização vertical de regulamentação e advertência (ausência, retro inadequada, avaria, depredação ou vandalismo), no Km 424.0
Apontamento número EIXO-default-2020.03339, Classe Reparo ou substituição de sinalização vertical de regulamentação e advertência (ausência, retro inadequada, avaria, depredação ou vandalismo), no Km 424.0
Apontamento número EIXO-default-2020.03340, Classe Reparo ou substituição de sinalização vertical de regulamentação e advertência (ausência, retro inadequada, avaria, depredação ou vandalismo), no Km 424.0
Apontamento número EIXO-default-2020.03433, Classe Reparo ou substituição de s

In [1]:
job=Job.objects.get(uuid='5d0fbbcb-5c7d-4864-9eb9-6d3d2577cb82')
job

<Job: [Eixo SP][matheus.negretti] - 2021-11-03 18:15:00+00:00 - 2021-11-13 10:35:00+00:00>

In [3]:
reportings=Reporting.history.model.objects.filter(job=job).distinct()
reportings.count()

193

In [6]:
reportings_now=Reporting.objects.filter(job=job)
reportings_now=[a.number for a in reportings_now]
len(reportings_now)

62

In [5]:
reportings_list=list(dict.fromkeys([a.number for a in reportings]))
len(reportings_list)

100

In [7]:
removed_reportings=[a for a in reportings_list if a not in reportings_now]
len(removed_reportings)

38

In [8]:
for a in removed_reportings:
    print(a)

EIXO-DRE-2021.21074
EIXO-DRE-2021.21055
EIXO-DRE-2021.21053
EIXO-DRE-2021.09579
EIXO-DRE-2021.09538
EIXO-DRE-2021.09510
EIXO-DRE-2021.09555
EIXO-DRE-2021.09525
EIXO-DRE-2021.09557
EIXO-DRE-2021.09564
EIXO-DRE-2021.09526
EIXO-DRE-2021.09529
EIXO-DRE-2021.04158
EIXO-DRE-2021.09509
EIXO-DRE-2021.09573
EIXO-DRE-2021.09553
EIXO-DRE-2021.04163
EIXO-DRE-2021.04159
EIXO-DRE-2021.09565
EIXO-DRE-2021.09484
EIXO-DRE-2021.09774
EIXO-DRE-2021.09517
EIXO-DRE-2021.09482
EIXO-DRE-2021.09462
EIXO-DRE-2021.09461
EIXO-DRE-2021.04170
EIXO-DRE-2021.09487
EIXO-DRE-2021.09468
EIXO-DRE-2021.04171
EIXO-DRE-2021.09483
EIXO-DRE-2021.04173
EIXO-DRE-2021.09472
EIXO-DRE-2021.09485
EIXO-DRE-2021.09470
EIXO-DRE-2021.04154
EIXO-DRE-2021.20739
EIXO-DRE-2021.04153
EIXO-DRE-2021.04152


In [4]:
job=Job.objects.filter(company=company, archived=False)
job.count()

729

In [5]:
for a in tqdm(job):
    a.save()

  0%|          | 0/729 [00:00<?, ?it/s]

In [3]:
reporting_list=['EIXO-default-2022.03188','EIXO-default-2022.03190', 'EIXO-default-2022.03191', 'EIXO-default-2022.03189', 'EIXO-default-2022.03194', 'EIXO-default-2022.03196', 'EIXO-default-2022.03230', 'EIXO-default-2022.03199']

In [7]:
for a in reporting_list:
    reporting=Reporting.objects.get(company=company, number=a)
#     print(reporting)
#     reporting.instance.save()
    for b in ReportingFile.history.model.objects.filter(reporting=reporting.uuid): 
        try:
            print(b)
            b.instance.save()
        except:
            print('AQUI')
            pass

[Eixo SP] - Constatação - 2022-01-27 11:12:14.562903+00:00 as of 2022-02-02 14:01:08.659276+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:14.562903+00:00 as of 2022-01-27 11:12:14.569574+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:15.894099+00:00 as of 2022-02-02 14:01:08.489975+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:15.894099+00:00 as of 2022-01-27 11:12:15.897681+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:16.589128+00:00 as of 2022-02-02 14:01:08.038668+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:16.589128+00:00 as of 2022-01-27 11:12:16.594740+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:15.115895+00:00 as of 2022-02-02 14:01:08.152767+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:15.115895+00:00 as of 2022-01-27 11:12:15.121132+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:19.132877+00:00 as of 2022-02-02 14:01:07.822725+00:00
[Eixo SP] - Constatação - 2022-01-27 11:12:19.132877+00:00 as of 2022-01-27 11:12:19.137048+00:00
[Eixo SP] - Constata

In [12]:
reportings=Reporting.objects.filter(company=company, road_name='SPI-083/191').only('road')
reportings.count()

3115

In [13]:
road_new=Road.objects.get(id=507)
road_new

<Road: 507: SPI-083/191>

In [14]:
reps=[]

for a in tqdm(reportings):
    a.road=road_new
    reps.append(a)

  0%|          | 0/3115 [00:00<?, ?it/s]

In [15]:
roads=bulk_update_with_history(reps, Reporting, user=user, use_django_bulk=True, batch_size=250)

In [90]:
report=MultipleDailyReport.objects.filter(company=company, uuid='7912929b-93f9-49d1-bf89-7196385b6551')
report.count()

1

In [92]:
firm=Firm.objects.get(company=company, name='SPD-LIMPEZA EPS')
subcompany=firm.subcompany
subcompany

<SubCompany: [097fe80a-4917-4fa1-8207-2de35c113599: Eixo SP] 22d9bb8d-9321-446c-a6f0-9349d16db8d9: Regional 01 - São Pedro>

In [93]:
for a in report:
    a.firm=firm
    a.save()

In [26]:
lista=['f762184c-1887-4119-add6-e5671cb07b47', '2fd1bb05-e096-43f8-9a94-06bbf127b65f', '578a2272-2cf3-42e4-988f-573a09290a30', '0c3312d2-2801-4d0e-ae35-d3b31b24742b', 'f71c97a4-1114-4c61-9f44-801d413bb905', '7e0b6121-ccd0-4b3a-9f1b-b34892c32d4e', '352e143c-c152-4000-bd4b-3b5bc26b9fd4', 'b96bf7de-909d-407d-b75a-d56ba27548d3', '7e74fe0b-926e-423b-acb0-64a1ea3b94e8', 'a4546f77-85ac-414a-95d0-ff3e1ec8b974', 'a3701b49-1d36-4830-bad3-9db693979d66', '38d7b9c8-b992-4053-9d4d-35671483a153', 'bc6bae58-c5da-4370-a04c-f4ea987c5ab6', '51215c19-e418-4dbf-9b46-b44fd912bbff', 'c43ca96a-c7c7-4c4f-8267-5b69435659be', 'f904fff4-9298-471a-b534-91279016632a', 'ba8e1129-6339-4f38-b89d-45b2e50df9ad', '7376ec3e-ccc7-45b6-ae9d-eec4d79dc039', '9bc62993-168a-44e5-9c3b-2a28b4511fcd']

In [27]:
reps=Reporting.objects.filter(uuid__in=lista).order_by('km')
reps.count()

19

In [30]:
for a in reps:
    try:
        print(a.number,"\t", a.km, "\t", a.form_data['artesp_code'])
    except Exception as e:
        print(a.number,"\t", a.km)

EIXO-default-2022.03193 	 357.26 	 32
EIXO-default-2022.03198 	 379.9 	 37
EIXO-default-2022.03200 	 386.95 	 39
EIXO-default-2022.03203 	 449.03 	 42
EIXO-default-2022.03234 	 459.7 	 40
EIXO-default-2022.03235 	 459.7 	 41
EIXO-default-2022.01559 	 461.74
EIXO-default-2022.01562 	 463.35
EIXO-default-2022.01561 	 463.42
EIXO-default-2022.01560 	 463.901
EIXO-default-2022.01565 	 464.76
EIXO-default-2022.01564 	 465.99
EIXO-default-2022.01563 	 467.9
EIXO-default-2022.03247 	 469.2 	 56
EIXO-default-2022.03236 	 480.2 	 42
EIXO-default-2022.03237 	 502.85 	 43
EIXO-default-2022.03238 	 504.8 	 44
EIXO-default-2022.03246 	 527.15 	 55
EIXO-default-2022.03239 	 529.05 	 45


In [6]:
lista=[
'EIXO-default-2022.03621',
'EIXO-default-2022.03491',
'EIXO-default-2022.03419',
'EIXO-default-2022.03418',
'EIXO-default-2022.03272',
'EIXO-default-2022.03271',
'EIXO-default-2022.03270',
'EIXO-default-2022.03269',
'EIXO-default-2022.03268',
'EIXO-default-2022.03261',
'EIXO-default-2022.03260',
'EIXO-default-2022.03165',
'EIXO-default-2022.03076',
'EIXO-default-2022.03075',
'EIXO-default-2022.03074',
'EIXO-default-2022.03073',
'EIXO-default-2022.03072',
'EIXO-default-2022.03071',
'EIXO-default-2022.02901',
'EIXO-default-2022.02900',
'EIXO-default-2022.02811',
'EIXO-default-2022.02693',
'EIXO-default-2022.02144',
'EIXO-default-2022.02143',
'EIXO-default-2022.01956',
'EIXO-default-2022.01699',
'EIXO-default-2022.01578',
'EIXO-default-2022.01569',
'EIXO-default-2022.01568',
'EIXO-default-2022.01567',
'EIXO-default-2022.01536',
'EIXO-default-2022.01504',
'EIXO-default-2022.01503',
'EIXO-default-2022.01478',
'EIXO-default-2022.01477',
'EIXO-default-2022.01322',
'EIXO-default-2022.01321',
'EIXO-default-2022.01320',
'EIXO-default-2022.00712',
'EIXO-default-2022.00190',
'EIXO-default-2022.00189'
]

In [7]:
reportings=Reporting.objects.filter(company=company, number__in=lista)
reportings.count()

40

In [10]:
for a in tqdm(reportings):
    if 'inspection_topics' in a.form_data and a.form_data['inspection_topics'] == ["8"]:
        del a.form_data['inspection_topics']
#         a.save()

In [15]:
reporting=Reporting.objects.get(uuid='7fac72c5-f2b1-4560-853e-2c72287d7052')
print(reporting.form_data['lots'])

{'lots': {'1': {'kms': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'road': 'SP-197', 'topics': {}}}, 'notes': 'DIN', 'road_code': 'Principal', 'executed_by': '2', 'road_system': '1', 'inspected_segments': None}


In [13]:
reportings=Reporting.objects.filter(company=company, occurrence_type__uuid='dee78e00-4481-48cd-b2ad-96861c5aadf2')
reportings.count()

978

In [32]:
for a in reportings:
    if 'inspection_topics' not in a.form_data:
        print(a.number)

EIXO-default-2021.11736
EIXO-default-2022.03073
EIXO-default-2021.02678
EIXO-default-2022.03261
EIXO-default-2022.01503
EIXO-default-2021.11550
EIXO-default-2021.11549
EIXO-default-2021.29829
EIXO-default-2022.02693
EIXO-default-2022.02811
EIXO-default-2022.00190
EIXO-default-2022.03165
EIXO-default-2022.03260
EIXO-default-2022.03269
EIXO-default-2022.03419
EIXO-default-2022.03491
EIXO-default-2022.03418
EIXO-default-2022.03074
EIXO-default-2022.03075
EIXO-default-2022.01320
EIXO-default-2022.01322
EIXO-default-2022.00712
EIXO-default-2022.01321
EIXO-default-2022.01477
EIXO-default-2022.01956
EIXO-default-2022.01699
EIXO-default-2022.01568
EIXO-default-2022.00189
EIXO-default-2022.03071
EIXO-default-2022.01478
EIXO-default-2022.03268
EIXO-default-2022.03272
EIXO-default-2022.03270
EIXO-default-2022.03271
EIXO-default-2022.03076
EIXO-default-2022.03621
EIXO-default-2022.02144
EIXO-default-2022.02143
EIXO-default-2022.02900
EIXO-default-2022.03072
EIXO-default-2022.01569
EIXO-default-202

In [30]:
for a in reportings:
    if 'lots' in a.form_data:
        for lot in a.form_data['lots'].keys():
            if a.form_data['lots'][lot]['topics'] == {}:
                print(a.number)
#             print(a.form_data['lots'][lot]['topics'])
#         try:
# #             if 'topics' not in a.form_data['lots'][next(iter(a.form_data['lots'].keys()))]:
#             print(a.form_data['lots'][next(iter(a.form_data['lots'].keys()))]['topics'])
# #                 pass
#         except Exception as e:
# #             print(a.number)
#             pass

EIXO-default-2022.03073
EIXO-default-2021.02678
EIXO-default-2022.03261
EIXO-default-2022.01503
EIXO-default-2021.11549
EIXO-default-2022.02693
EIXO-default-2022.02811
EIXO-default-2022.00190
EIXO-default-2022.03165
EIXO-default-2022.03260
EIXO-default-2022.03269
EIXO-default-2022.03419
EIXO-default-2022.03491
EIXO-default-2022.03418
EIXO-default-2022.03074
EIXO-default-2022.03075
EIXO-default-2022.01320
EIXO-default-2022.01322
EIXO-default-2022.00712
EIXO-default-2022.01321
EIXO-default-2022.01477
EIXO-default-2022.01956
EIXO-default-2022.01699
EIXO-default-2022.01568
EIXO-default-2022.00189
EIXO-default-2022.03071
EIXO-default-2022.01478
EIXO-default-2022.03268
EIXO-default-2022.03272
EIXO-default-2022.03272
EIXO-default-2022.03270
EIXO-default-2022.03270
EIXO-default-2022.03271
EIXO-default-2022.03271
EIXO-default-2022.03076
EIXO-default-2022.03621
EIXO-default-2022.02144
EIXO-default-2022.02143
EIXO-default-2022.02900
EIXO-default-2022.03072
EIXO-default-2022.01569
EIXO-default-202

In [2]:
rdo_dict = {'EIXO-RDO-2022.00363': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00195': 'Roçada MAN 4 - Reg.03',
'EIXO-RDO-2022.00366': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00276': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00314': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00359': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00324': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00274': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00364': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00368': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00241': 'Roçada MAN 4 - Reg.03',
'EIXO-RDO-2022.00233': 'Roçada MAN 7 - Reg.03',
'EIXO-RDO-2022.00151': 'Roçada MAN 4 - Reg.03',
'EIXO-RDO-2022.00158': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00231': 'Roçada MAN 7 - Reg.03',
'EIXO-RDO-2022.00232': 'Roçada MAN 7 - Reg.03',
'EIXO-RDO-2022.00355': 'Roçada MAN 4 - Reg.03',
'EIXO-RDO-2022.00275': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00291': 'Roçada MAN 4 - Reg.03',
'EIXO-RDO-2022.00258': 'Roçada MAN 7 - Reg.03',
'EIXO-RDO-2022.00360': 'Roçada MAN 2 - Reg.03',
'EIXO-RDO-2022.00290': 'Roçada MAN 4 - Reg.03'}

In [7]:
for rdo, rdo_firm in rdo_dict.items():
    rdo=MultipleDailyReport.objects.get(company=company, number=rdo)
    try:
        rdo.firm=Firm.objects.get(company=company, name=rdo_firm)
        rdo.save()
    except Exception as e:
        print(rdo.number)

EIXO-RDO-2022.00363
EIXO-RDO-2022.00366
EIXO-RDO-2022.00359
EIXO-RDO-2022.00324
EIXO-RDO-2022.00364
EIXO-RDO-2022.00275
EIXO-RDO-2022.00291
EIXO-RDO-2022.00258


In [15]:
rdo_delete=['EIXO-RDO-2022.01067', 'EIXO-RDO-2022.01070']

In [16]:
for a in rdo_delete:
    rdo=MultipleDailyReport.objects.get(company=company, number=a)
    rdo.delete()

In [8]:
rdo_dict={'EIXO-RDO-2022.01074': 'EQUIPE VOLANTE 2 - ITIRAPINA',
            'EIXO-RDO-2022.01073': 'EQUIPE VOLANTE 2 - ITIRAPINA',
            'EIXO-RDO-2022.01018': 'EQUIPE VOLANTE 2 - ITIRAPINA',
            'EIXO-RDO-2022.01067': 'EQUIPE VOLANTE 2 - ITIRAPINA',
            'EIXO-RDO-2022.01070': 'EQUIPE VOLANTE 2 - ITIRAPINA',
            'EIXO-RDO-2022.01121': 'LIMP DRENAGEM 1 - ITIRAPINA',
            'EIXO-RDO-2022.00874': 'DESPRAGUEJAMENTO ITIRAPINA',
            'EIXO-RDO-2022.01186': 'PODA DE ÁRVORES ITIRAPINA',
            'EIXO-RDO-2022.01065': 'PODA DE ÁRVORES ITIRAPINA',
            'EIXO-RDO-2022.01187': 'PODA DE ÁRVORES ITIRAPINA',
            'EIXO-RDO-2022.01238': 'PODA DE ÁRVORES ITIRAPINA',
            'EIXO-RDO-2022.01057': 'PODA DE ÁRVORES ITIRAPINA',
            'EIXO-RDO-2022.01280': 'PODA DE ÁRVORES ITIRAPINA',
            'EIXO-RDO-2022.00054': 'DEFENSA METÁLICA ITIRAPINA',
            'EIXO-RDO-2022.00045': 'DEFENSA METÁLICA ITIRAPINA',
            'EIXO-RDO-2022.00105': 'DEFENSA METÁLICA ITIRAPINA',
            'EIXO-RDO-2022.00077': 'DEFENSA METÁLICA ITIRAPINA'}

In [12]:
for rdo, rdo_firm in rdo_dict.items():
    rdo=MultipleDailyReport.objects.get(company=company, number=rdo)
    try:
        rdo.firm=Firm.objects.get(company=company, name=rdo_firm)
        rdo.save()
    except Exception as e:
        print(rdo.number, e)

EIXO-RDO-2022.01067 duplicate key value violates unique constraint "daily_reports_multipleda_created_by_id_firm_id_da_394697a2_uniq"
DETAIL:  Key (created_by_id, firm_id, date)=(af024ef5-d2d5-44e5-9c08-f50f6990cc30, c8ef45cf-0750-49de-9d43-e0a9bb4da5c7, 2022-04-18) already exists.

EIXO-RDO-2022.01070 duplicate key value violates unique constraint "daily_reports_multipleda_created_by_id_firm_id_da_394697a2_uniq"
DETAIL:  Key (created_by_id, firm_id, date)=(af024ef5-d2d5-44e5-9c08-f50f6990cc30, c8ef45cf-0750-49de-9d43-e0a9bb4da5c7, 2022-04-19) already exists.



In [4]:
rdo=MultipleDailyReport.objects.get(number='EIXO-RDO-2022.08425')
rdo

<MultipleDailyReport: [Eixo SP] bac1cace-9664-433b-9c1d-1a13b8156e56 - 2022-08-03>

In [2]:
firm=Firm.objects.get(name='MULTIFUNCIONAL ITIRAPINA')
firm

<Firm: [097fe80a-4917-4fa1-8207-2de35c113599: Eixo SP] 41fa3c0b-e28b-42d8-8d22-e4b916cd34ad: MULTIFUNCIONAL ITIRAPINA>

In [3]:
rdo.firm=firm
# rdo.save()

In [39]:
reps=Reporting.objects.filter(company=company, lot__isnull=True).order_by('road_name', 'created_at').exclude(occurrence_type__occurrence_kind='2')
reps.count()

475

In [44]:
print(f'Número\tRodovia\tKm\tCriado em\tCriado por')
for a in reps:
    print(f'{a.number}\t{a.road_name}\t{a.km}\t{a.created_at.strftime("%d/%m/%Y")}\t{a.created_by.get_full_name()}')

Número	Rodovia	Km	Criado em	Criado por
EIXO-default-2022.20328	SP000	0.0	04/06/2022	Integração TOR
EIXO-PAV-2020.00879	SP-191	116.5	15/07/2020	Luiz Oliveira
EIXO-PAV-2020.00884	SP-191	116.1	15/07/2020	Luiz Oliveira
EIXO-PAV-2020.00885	SP-191	116.3	15/07/2020	Luiz Oliveira
EIXO-PAV-2020.00886	SP-191	116.5	15/07/2020	Luiz Oliveira
EIXO-PAV-2020.00887	SP-191	116.6	15/07/2020	Luiz Oliveira
EIXO-PAV-2020.01123	SP-191	74.53	22/07/2020	Luiz Oliveira
EIXO-PAV-2020.01124	SP-191	74.6	22/07/2020	Luiz Oliveira
EIXO-DRE-2021.12941	SP-191	74.65	01/07/2021	Rosiane Costa
EIXO-FDO-2020.02414	SP-197	20.0	19/08/2020	Anderson Sousa
EIXO-default-2021.04872	SP-197	249.37	23/02/2021	Luis Carlos Soares
EIXO-default-2022.31444	SP-197	1000.0	04/08/2022	David Sardinha
EIXO-default-2022.31456	SP-197	4000.0	04/08/2022	David Sardinha
EIXO-default-2022.39314	SP-197	20.0	22/09/2022	Guilherme de Castro Silva
EIXO-default-2022.39315	SP-197	20.0	22/09/2022	Guilherme de Castro Silva
EIXO-default-2022.39316	SP-197	20.0	22

In [20]:
reps_edit=[]
counter=0

for a in tqdm(reps):
    
    data = {"data": {"km": a.km}}
    
    try:
        lot=jsonLogic(a.road.lot_logic, data)
        if lot is None:
            counter+=1
        else:
            a.lot=lot
            reps_edit.append(a)
    except Exception:
        counter+=1

  0%|          | 0/4161 [00:00<?, ?it/s]

In [22]:
# reporting_edit=bulk_update_with_history(reps_edit, Reporting, user=user, use_django_bulk=True, batch_size=100)

In [53]:
user=UserInCompany.history.model.objects.filter(user__username='edmar.werner').order_by('history_date')

In [54]:
for a in user:
    print(a.permissions, a.history_date, a.history_user, a.history_type)

['Neovia Ponte Serrada', 'Neovia Oeste'] - Encarregado 2022-09-19 12:09:33.668706+00:00 carlos.dahora - Carlos da Hora +
['Neovia Juquitiba', 'Neovia Rancho Queimado 080/2021', 'Brusque', 'Neovia Vacaria', 'Arteris Litoral Sul', 'SETA', 'Neovia Planalto Sul', 'Neovia Lev. BR277', 'Fratello MT-220 Porto dos Gaúchos', 'Dalba L14', 'Arteris Via Paulista - SISOBRAS', 'Neovia - TESTE', 'SETA', 'Arteris Régis Bittencourt - Conserva', 'VF Gomes - BR158', 'Neovia Teste', 'Neovia Florianópolis', 'PoC SISOAE', 'Neovia Tijucas', 'Campina Grande', 'Teste Migração', 'Neovia Araquari 886/2020', 'Porto Itapoá', 'Neovia Lages 421/2014', 'Teste Concessionárias', 'Neovia Regis', 'Arteris Planalto Sul - Con', 'TESTE - CONCESSIONÁRIA', 'ECOPISTAS', 'VF Gomes - 455', 'Teste Integração - OptGIS', 'VF Gomes - 350', 'MTSUL - Cláudia', 'Demonstração - SISOBRAS', 'Neovia BR-285', 'Fratello MT-130 Paranatinga', 'POC-Projeto APR', 'Afirma Engenharia Viária', 'CV Seg - Via Sul', "Neovia Pérola d'Oeste", 'Demo Conc

In [2]:
lista=["EIXO-default-2022.42574",
"EIXO-default-2022.42573",
"EIXO-default-2022.42572",
"EIXO-default-2022.42555",
"EIXO-default-2022.42554",
"EIXO-default-2022.42553",
"EIXO-default-2022.42552"]

In [3]:
company=Company.objects.get(name='Eixo SP')
user=User.objects.get(username='rlcs')

In [4]:
reps=Reporting.objects.filter(company=company, number__in=lista)
reps.count()

7

In [7]:
reps_edit=[]
counter=0

for a in tqdm(reps):
    del a.form_data['artesp_code']
    if 'nc_artesp' in a.form_data:
        del a.form_data['nc_artesp']
        counter+=1
    reps_edit.append(a)

  0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
reporting_edit=bulk_update_with_history(reps_edit, Reporting, user=user, use_django_bulk=True, batch_size=100)